<a href="https://colab.research.google.com/github/DIYDataScienceYT/KaggleCodes/blob/main/1_WritingQualityBaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [ ]:
# General library imports:-
# displaying styling elements (HTML, Markdown..)
from IPython.display import display_html, clear_output, Markdown;
# trigger manual garbage collection cycle
from gc import collect;
# working with dataframes
import pandas as pd;
# numerical and mathematical operations
import numpy as np;
# makes copy
from copy import deepcopy;
# frequence counter
from collections import Counter
